In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
# imports

import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor


In [3]:
# Read the data
X = pd.read_csv('../input/home-data-for-ml-course/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/home-data-for-ml-course/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)



In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuilt    

In [5]:
object_cols = [cname for cname in X.columns if X[cname].dtype == "object"]

# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Alley', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('MasVnrType', 3),
 ('GarageFinish', 3),
 ('PavedDrive', 3),
 ('PoolQC', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('BsmtQual', 4),
 ('BsmtCond', 4),
 ('BsmtExposure', 4),
 ('KitchenQual', 4),
 ('Fence', 4),
 ('MiscFeature', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Electrical', 5),
 ('FireplaceQu', 5),
 ('GarageQual', 5),
 ('GarageCond', 5),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('BsmtFinType1', 6),
 ('BsmtFinType2', 6),
 ('Heating', 6),
 ('GarageType', 6),
 ('SaleCondition', 6),
 ('Functional', 7),
 ('Condition2', 8),
 ('HouseStyle', 8),
 ('RoofMatl', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [6]:
# analise dos missings
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
for col in cols_with_missing:
    print(f"{col}: {X[col].dtype} qtd nulo: {X[col].isnull().sum()} ratio: {round((X[col].isnull().sum() / len(X))*100,2)}")

LotFrontage: float64 qtd nulo: 259 ratio: 17.74
Alley: object qtd nulo: 1369 ratio: 93.77
MasVnrType: object qtd nulo: 872 ratio: 59.73
MasVnrArea: float64 qtd nulo: 8 ratio: 0.55
BsmtQual: object qtd nulo: 37 ratio: 2.53
BsmtCond: object qtd nulo: 37 ratio: 2.53
BsmtExposure: object qtd nulo: 38 ratio: 2.6
BsmtFinType1: object qtd nulo: 37 ratio: 2.53
BsmtFinType2: object qtd nulo: 38 ratio: 2.6
Electrical: object qtd nulo: 1 ratio: 0.07
FireplaceQu: object qtd nulo: 690 ratio: 47.26
GarageType: object qtd nulo: 81 ratio: 5.55
GarageYrBlt: float64 qtd nulo: 81 ratio: 5.55
GarageFinish: object qtd nulo: 81 ratio: 5.55
GarageQual: object qtd nulo: 81 ratio: 5.55
GarageCond: object qtd nulo: 81 ratio: 5.55
PoolQC: object qtd nulo: 1453 ratio: 99.52
Fence: object qtd nulo: 1179 ratio: 80.75
MiscFeature: object qtd nulo: 1406 ratio: 96.3


In [7]:
# analise dos missings do X_Test
cols_with_missing = [col for col in X_test_full.columns if X_test_full[col].isnull().any()] 
for col in cols_with_missing:
    print(f"{col}: {X_test_full[col].dtype} qtd nulo: {X_test_full[col].isnull().sum()} ratio: {round((X_test_full[col].isnull().sum() / len(X_test_full))*100,2)}")

MSZoning: object qtd nulo: 4 ratio: 0.27
LotFrontage: float64 qtd nulo: 227 ratio: 15.56
Alley: object qtd nulo: 1352 ratio: 92.67
Utilities: object qtd nulo: 2 ratio: 0.14
Exterior1st: object qtd nulo: 1 ratio: 0.07
Exterior2nd: object qtd nulo: 1 ratio: 0.07
MasVnrType: object qtd nulo: 894 ratio: 61.27
MasVnrArea: float64 qtd nulo: 15 ratio: 1.03
BsmtQual: object qtd nulo: 44 ratio: 3.02
BsmtCond: object qtd nulo: 45 ratio: 3.08
BsmtExposure: object qtd nulo: 44 ratio: 3.02
BsmtFinType1: object qtd nulo: 42 ratio: 2.88
BsmtFinSF1: float64 qtd nulo: 1 ratio: 0.07
BsmtFinType2: object qtd nulo: 42 ratio: 2.88
BsmtFinSF2: float64 qtd nulo: 1 ratio: 0.07
BsmtUnfSF: float64 qtd nulo: 1 ratio: 0.07
TotalBsmtSF: float64 qtd nulo: 1 ratio: 0.07
BsmtFullBath: float64 qtd nulo: 2 ratio: 0.14
BsmtHalfBath: float64 qtd nulo: 2 ratio: 0.14
KitchenQual: object qtd nulo: 1 ratio: 0.07
Functional: object qtd nulo: 2 ratio: 0.14
FireplaceQu: object qtd nulo: 730 ratio: 50.03
GarageType: object qtd n

In [8]:
# ajuste das colunas com missing
# Removendo colunas com mais de 50% de missing
X.drop(['Alley','MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis=1, inplace=True)
X_test_full.drop(['Alley','MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis=1, inplace=True)


In [9]:
# ajuste das colunas com missing
# Colunas numéricas com missing transformadas com valor zero
colNumMissing = ['LotFrontage','MasVnrArea', 'GarageYrBlt', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', 
                 'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea']

# zerando quando indica valor construído
for col in colNumMissing:
    X[col] = X[col].fillna(0)
    X_test_full[col] = X_test_full[col].fillna(0)

# colocando para infinito quando indica valor não construído
colNumMissing = ['BsmtUnfSF']
for col in colNumMissing:
    X[col] = X[col].fillna(9999999)
    X_test_full[col] = X_test_full[col].fillna(9999999)    
    
# Colunas string com missing transformadas caso a caso
# MSZoning: não existe indicação do que NA significa. Tem 4 linhas com NA no X_test_full. => vou transformar na string XX
# BsmtQual: NA significa No Basement => vou transformar na string XX
# BsmtCond: NA significa No Basement => vou transformar na string XX
# BsmtExposure: NA significa No Basement => vou transformar na string XX
# BsmtFinType1: NA significa No Basement => vou transformar na string XX
# BsmtFinType2: NA significa No Basement => vou transformar na string XX
# Electrical: não existe indicação do que NA significa. Tem apenas uma linha com NA. => vou transformar na string XX
# GarageType: NA significa No Garage => vou transformar na string XX
# GarageFinish: NA significa No Garage => vou transformar na string XX
# GarageQual: NA significa No Garage => vou transformar na string XX
# GarageCond: NA significa No Garage => vou transformar na string XX
# Utilities: não existe indicação do que NA significa. Tem 2 linhas com NA no X_test_full. => vou transformar na string XX
# Exterior1st: não existe indicação do que NA significa. Tem 1 linha com NA no X_test_full. => vou transformar na string XX
# Exterior2nd: não existe indicação do que NA significa. Tem 1 linha com NA no X_test_full. => vou transformar na string XX
# KitchenQual: não existe indicação do que NA significa. Tem 1 linha com NA no X_test_full. => vou transformar na string XX
# Functional: não existe indicação do que NA significa. Tem 1 linha com NA no X_test_full. => vou transformar na string XX
# SaleType: não existe indicação do que NA significa. Tem 1 linha com NA no X_test_full. => vou transformar na string XX

colStrMissing = ['MSZoning',
                 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1','BsmtFinType2', 
                 'Electrical', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
                 'Utilities', 'Exterior1st','Exterior2nd', 'KitchenQual', 'Functional', 'SaleType']

for col in colStrMissing:
    X[col] = X[col].fillna('XX')
    X_test_full[col] = X_test_full[col].fillna('XX')



In [10]:
# verificando se ainda existem missing
cols_with_missing = [col for col in X.columns if X[col].isnull().any()] 
for col in cols_with_missing:
    print(f"{col}: {X[col].dtype} qtd nulo: {X[col].isnull().sum()} ratio: {round((X[col].isnull().sum() / len(X))*100,2)}")

In [11]:
# verificando se ainda existem missing
# analise dos missings do X_Test
cols_with_missing = [col for col in X_test_full.columns if X_test_full[col].isnull().any()] 
for col in cols_with_missing:
    print(f"{col}: {X_test_full[col].dtype} qtd nulo: {X_test_full[col].isnull().sum()} ratio: {round((X_test_full[col].isnull().sum() / len(X_test_full))*100,2)}")

In [12]:
# Aplicar Ordinal Encoding nas colunas que faz sentido
# Street: Pave, Grvl
# LotShape: Reg, IR1, IR2, IR3
# Utilities: AllPub, NoSewr, NoSeWa, ELO
# ExterQual, ExterCond, BsmtQual, BsmtCond, HeatingQC, KitchenQual, GarageQual, GarageCond: Ex, Gd, TA, Fa, Po, XX
# BsmtExposure: Gd, Av, Mn, No, XX
# BsmtFinType1, BsmtFinType2: GLQ, ALQ, BLQ, Rec, LwQ, Unf, XX
# CentralAir: Y, N, XX
# Functional: Typ, Min1, Min2, Mod, Maj1, Maj2, Sev, Sal, XX
# GarageFinish: Fin, RFn, Unf, XX


ordinal_encoder_LotShape = OrdinalEncoder(categories=[
                                                    ['Pave','Grvl', 'XX'],                                              # Street
                                                    ['Reg', 'IR1', 'IR2', 'IR3', 'XX'],                                 # LotShape
                                                    ['AllPub', 'NoSewr', 'NoSeWa', 'ELO', 'XX'],                        # Utilities
                                                    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'XX'],                               # ExterQual
                                                    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'XX'],                               # ExterCond
                                                    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'XX'],                               # BsmtQual
                                                    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'XX'],                               # BsmtCond
                                                    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'XX'],                               # HeatingQC
                                                    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'XX'],                               # KitchenQual
                                                    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'XX'],                               # GarageQual
                                                    ['Ex', 'Gd', 'TA', 'Fa', 'Po', 'XX'],                               # GarageCond
                                                    ['Gd', 'Av', 'Mn', 'No', 'XX'],                                     # BsmtExposure
                                                    ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'XX'],                   # BsmtFinType1
                                                    ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LwQ', 'Unf', 'XX'],                   # BsmtFinType2
                                                    ['Y','N','XX'],                                                     # CentralAir
                                                    ['Typ', 'Min1', 'Min2', 'Mod', 'Maj1', 'Maj2', 'Sev', 'Sal', 'XX'], # Functional
                                                    ['Fin', 'RFn', 'Unf', 'XX']                                         # GarageFinish
                                                   ],
                                          encoded_missing_value = 999)

ordinal_columns = ['Street',
                   'LotShape',
                   'Utilities',
                   'ExterQual','ExterCond','BsmtQual', 'BsmtCond', 'HeatingQC','KitchenQual', 'GarageQual', 'GarageCond',
                   'BsmtExposure', 
                   'BsmtFinType1', 'BsmtFinType2',
                   'CentralAir',
                   'Functional',
                   'GarageFinish']

X[ordinal_columns] = ordinal_encoder_LotShape.fit_transform(X[ordinal_columns])

X_test_full[ordinal_columns] = ordinal_encoder_LotShape.transform(X_test_full[ordinal_columns])


In [13]:
X['LotShape']

Id
1       0.0
2       0.0
3       1.0
4       1.0
5       1.0
       ... 
1456    0.0
1457    0.0
1458    0.0
1459    0.0
1460    0.0
Name: LotShape, Length: 1460, dtype: float64

In [14]:
X[X['Utilities'] > 0]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
945,20,RL,0.0,14375,0.0,1.0,Lvl,2.0,CulDSac,Gtl,...,118,0,0,233,0,0,1,2009,COD,Abnorml


In [15]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, 
                                                                test_size=0.2,
                                                                random_state=0)

In [16]:
# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train_full)
X_valid = pd.get_dummies(X_valid_full)
X_test = pd.get_dummies(X_test_full)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [17]:
# Coloca False nas colunas do X_valid e X_test que forem NA
# Essas colunas foram criadas pelo align e se elas estão com NA, significa que não existiam e foram criadas para manter o shape

# analise dos missings do X_Valid
cols_with_missing = [col for col in X_valid.columns if X_valid[col].isnull().any()] 
for col in cols_with_missing:
    print(f"{col}: {X_valid[col].dtype} qtd nulo: {X_valid[col].isnull().sum()} ratio: {round((X_valid[col].isnull().sum() / len(X_valid))*100,2)}")


LandSlope_Sev: float64 qtd nulo: 292 ratio: 100.0
Neighborhood_Blueste: float64 qtd nulo: 292 ratio: 100.0
Condition1_RRNe: float64 qtd nulo: 292 ratio: 100.0
Condition2_PosA: float64 qtd nulo: 292 ratio: 100.0
Condition2_RRAe: float64 qtd nulo: 292 ratio: 100.0
RoofStyle_Shed: float64 qtd nulo: 292 ratio: 100.0
RoofMatl_Membran: float64 qtd nulo: 292 ratio: 100.0
RoofMatl_Metal: float64 qtd nulo: 292 ratio: 100.0
RoofMatl_Roll: float64 qtd nulo: 292 ratio: 100.0
RoofMatl_WdShake: float64 qtd nulo: 292 ratio: 100.0
Exterior1st_AsphShn: float64 qtd nulo: 292 ratio: 100.0
Exterior1st_CBlock: float64 qtd nulo: 292 ratio: 100.0
Exterior1st_ImStucc: float64 qtd nulo: 292 ratio: 100.0
Exterior1st_Stone: float64 qtd nulo: 292 ratio: 100.0
Exterior2nd_CBlock: float64 qtd nulo: 292 ratio: 100.0
Exterior2nd_Other: float64 qtd nulo: 292 ratio: 100.0
Foundation_Stone: float64 qtd nulo: 292 ratio: 100.0
Heating_Floor: float64 qtd nulo: 292 ratio: 100.0
Heating_Grav: float64 qtd nulo: 292 ratio: 100

In [18]:
for col in cols_with_missing:
    X_valid[col] = X_valid[col].fillna(False)
    X_test[col] = X_test[col].fillna(False)

In [19]:
# analise dos missings do X_Test
cols_with_missing = [col for col in X_test.columns if X_test[col].isnull().any()] 
for col in cols_with_missing:
    print(f"{col}: {X_test[col].dtype} qtd nulo: {X_test[col].isnull().sum()} ratio: {round((X_test[col].isnull().sum() / len(X_test))*100,2)}")

HouseStyle_2.5Fin: float64 qtd nulo: 1459 ratio: 100.0


In [20]:
for col in cols_with_missing:
    X_test[col] = X_test[col].fillna(False)

In [21]:
# Verifica se alguma coluna continua com missing em X_valid e X_test
# Essas colunas foram criadas pelo align e se elas estão com NA, significa que não existiam e foram criadas para manter o shape

# analise dos missings do X_Valid
cols_with_missing = [col for col in X_valid.columns if X_valid[col].isnull().any()] 
for col in cols_with_missing:
    print(f"X_Valid {col}: {X_valid[col].dtype} qtd nulo: {X_valid[col].isnull().sum()} ratio: {round((X_valid[col].isnull().sum() / len(X_valid))*100,2)}")

    
# analise dos missings do X_Test
cols_with_missing = [col for col in X_test.columns if X_test[col].isnull().any()] 
for col in cols_with_missing:
    print(f"X_Test {col}: {X_test[col].dtype} qtd nulo: {X_test[col].isnull().sum()} ratio: {round((X_test[col].isnull().sum() / len(X_test))*100,2)}")

In [22]:
X_train

,MSSubClass,LotFrontage,LotArea,Street,LotShape,Utilities,OverallQual,OverallCond,YearBuilt,YearRemodAdd,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,0.0,0.0,0.0,9,5,2007,2007,...,False,True,False,False,False,False,False,False,False,True
871,20,60.0,6600,0.0,0.0,0.0,5,5,1962,1962,...,False,False,False,True,False,False,False,False,True,False
93,30,80.0,13360,0.0,1.0,0.0,5,7,1921,2006,...,False,False,False,True,False,False,False,False,True,False
818,20,0.0,13265,0.0,1.0,0.0,8,5,2002,2002,...,False,False,False,True,False,False,False,False,True,False
303,20,118.0,13704,0.0,1.0,0.0,7,5,2001,2002,...,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,60,82.0,9430,0.0,0.0,0.0,8,5,1999,1999,...,False,False,False,True,False,False,False,False,True,False
836,20,60.0,9600,0.0,0.0,0.0,4,7,1950,1995,...,False,False,False,True,False,False,False,False,True,False
1217,90,68.0,8930,0.0,0.0,0.0,6,5,1978,1978,...,False,False,False,True,False,False,False,False,True,False


In [23]:
X_valid

,MSSubClass,LotFrontage,LotArea,Street,LotShape,Utilities,OverallQual,OverallCond,YearBuilt,YearRemodAdd,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
530,20,0.0,32668,0.0,1.0,0.0,6,3,1957,1975,...,False,False,False,True,False,False,True,False,False,False
492,50,79.0,9490,0.0,0.0,0.0,6,7,1941,1950,...,False,False,False,True,False,False,False,False,True,False
460,50,0.0,7015,0.0,1.0,0.0,5,4,1950,1950,...,False,False,False,True,False,False,False,False,True,False
280,60,83.0,10005,0.0,0.0,0.0,7,5,1977,1977,...,False,False,False,True,False,False,False,False,True,False
656,160,21.0,1680,0.0,0.0,0.0,6,5,1971,1971,...,False,False,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,120,32.0,10846,0.0,1.0,0.0,8,5,1993,1993,...,False,False,False,False,False,False,False,False,True,False
441,20,105.0,15431,0.0,0.0,0.0,10,5,2008,2008,...,False,False,False,True,False,False,False,False,True,False
1388,50,60.0,8520,0.0,0.0,0.0,6,7,1916,1950,...,False,False,False,False,False,False,False,True,False,False


In [24]:
X_test

,MSSubClass,LotFrontage,LotArea,Street,LotShape,Utilities,OverallQual,OverallCond,YearBuilt,YearRemodAdd,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,80.0,11622,0.0,0.0,0.0,5,6,1961,1961,...,False,False,False,True,False,False,False,False,True,False
1462,20,81.0,14267,0.0,1.0,0.0,6,6,1958,1958,...,False,False,False,True,False,False,False,False,True,False
1463,60,74.0,13830,0.0,1.0,0.0,5,5,1997,1998,...,False,False,False,True,False,False,False,False,True,False
1464,60,78.0,9978,0.0,1.0,0.0,6,6,1998,1998,...,False,False,False,True,False,False,False,False,True,False
1465,120,43.0,5005,0.0,1.0,0.0,8,5,1992,1992,...,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,21.0,1936,0.0,0.0,0.0,4,7,1970,1970,...,False,False,False,True,False,False,False,False,True,False
2916,160,21.0,1894,0.0,0.0,0.0,4,5,1970,1970,...,False,False,False,True,True,False,False,False,False,False
2917,20,160.0,20000,0.0,0.0,0.0,5,7,1960,1996,...,False,False,False,True,True,False,False,False,False,False


In [25]:
# Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(mean_absolute_error(y_valid, preds))

17159.995034246575


Numero anterior sem fazer ordinal classification 17573.964143835616

In [26]:
# Prediction
preds_test = model.predict(X_test)

In [27]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)